In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
from docx import Document
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas



class ResumeBuilder:
    def __init__(self, root):
        self.root = root
        self.root.title("Resume checker")
        self.root.geometry("400x600")

        
        self.create_input_fields()

        # Create buttons
        self.generate_pdf_button = tk.Button(root, text="Generate PDF", command=self.generate_pdf)
        self.generate_pdf_button.pack()

        self.generate_doc_button = tk.Button(root, text="Generate DOCX", command=self.generate_doc)
        self.generate_doc_button.pack()

    def create_input_fields(self):
        labels = ["Name", "Email", "Phone", "Address", "Summary"]

        self.entries = {}
        for label in labels:
            lbl = tk.Label(self.root, text=label)
            lbl.pack()
            entry = tk.Entry(self.root)
            entry.pack()
            self.entries[label] = entry

        # Experience, Education, Projects, Certificates, Skills
        self.experience_list = []
        self.education_list = []
        self.projects_list = []
        self.certificates_list = []
        self.skills_list = []

        self.create_list_input("Experience", self.experience_list)
        self.create_list_input("Education", self.education_list)
        self.create_list_input("Projects", self.projects_list)
        self.create_list_input("Certificates", self.certificates_list)
        self.create_list_input("Skills", self.skills_list)

    def create_list_input(self, label, list_storage):
        lbl = tk.Label(self.root, text=label)
        lbl.pack()
        entry = tk.Entry(self.root)
        entry.pack()
        add_button = tk.Button(self.root, text=f"Add {label[:-1]}", command=lambda: self.add_to_list(entry, list_storage))
        add_button.pack()

    def add_to_list(self, entry, list_storage):
        item = entry.get().strip()
        if item:
            list_storage.append(item)
            entry.delete(0, tk.END)
            messagebox.showinfo("Success", f"{item} added to {list_storage[-1]}.")
        else:
            messagebox.showerror("Input Error", "Please enter a valid item.")

    def generate_pdf(self):
        data = self.get_input_data()
        if not data:
            return
        
        filename = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        if filename:
            c = canvas.Canvas(filename, pagesize=letter)
            c.drawString(100, 750, f"Name: {data['Name']}")
            c.drawString(100, 730, f"Email: {data['Email']}")
            c.drawString(100, 710, f"Phone: {data['Phone']}")
            c.drawString(100, 690, f"Address: {data['Address']}")
            c.drawString(100, 670, f"Summary: {data['Summary']}")
            self.draw_list(c, "Experience", data['Experience'], 650)
            self.draw_list(c, "Education", data['Education'], 550)
            self.draw_list(c, "Projects", data['Projects'], 450)
            self.draw_list(c, "Certificates", data['Certificates'], 350)
            self.draw_list(c, "Skills", data['Skills'], 250)
            c.save()
            messagebox.showinfo("Success", "PDF generated successfully!")

    def generate_doc(self):
        data = self.get_input_data()
        if not data:
            return

        filename = filedialog.asksaveasfilename(defaultextension=".docx", filetypes=[("Word files", "*.docx")])
        if filename:
            doc = Document()
            doc.add_heading('Resume', level=1)
            doc.add_paragraph(f"Name: {data['Name']}")
            doc.add_paragraph(f"Email: {data['Email']}")
            doc.add_paragraph(f"Phone: {data['Phone']}")
            doc.add_paragraph(f"Address: {data['Address']}")
            doc.add_paragraph(f"Summary: {data['Summary']}")
            
            self.add_list_to_doc(doc, "Experience", data['Experience'])
            self.add_list_to_doc(doc, "Education", data['Education'])
            self.add_list_to_doc(doc, "Projects", data['Projects'])
            self.add_list_to_doc(doc, "Certificates", data['Certificates'])
            self.add_list_to_doc(doc, "Skills", data['Skills'])

            doc.save(filename)
            messagebox.showinfo("Success", "DOCX generated successfully!")

    def draw_list(self, c, title, items, y_start):
        c.drawString(100, y_start, title + ":")
        y = y_start - 20
        for item in items:
            c.drawString(100, y, item)
            y -= 20

    def add_list_to_doc(self, doc, title, items):
        doc.add_heading(title, level=2)
        for item in items:
            doc.add_paragraph(item)

    def get_input_data(self):
        data = {label: entry.get().strip() for label, entry in self.entries.items()}
        data['Experience'] = self.experience_list
        data['Education'] = self.education_list
        data['Projects'] = self.projects_list
        data['Certificates'] = self.certificates_list
        data['Skills'] = self.skills_list

        if not all(data.values()):
            messagebox.showerror("Input Error", "Please fill in all fields.")
            return None
        return data

if __name__ == "__main__":
    root = tk.Tk()
    app = ResumeBuilder(root)
    root.mainloop()
